# Projeto 2 - Sistema de Recomendação para Aplicativos de Contabilidade

Neste projeto o objetivo é construir um sistema de recomendação com a arquitetura Graph Attention Network – GAT através do mecanismo de atenção em uma rede convolucional de grafos. O projeto é em Linguagem Python com PyTorch e PyTorch Geometric.Ao final do projeto o sistema será capaz de recomendar aplicativos de Contabilidade.

## 1. Carga e Instalação dos Pacotes

In [2]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da linguagem Python usada neste Jupyter Notebook é: ', python_version())

Versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [3]:
# Instalação Pytorch
!pip install -q torch==2.0.0

In [4]:
# Instalação Pytorch Geometric
!pip install -q torch_geometric==2.3.1

In [5]:
# Carga dos Pacotes
# Manipulação de Dados
import pandas as pd
import numpy as np

# Pytorch
import torch
import torch.nn.functional as F

# Pytorch-Geometric
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.utils import to_undirected
from torch_geometric.loader import DataLoader

#Scikit-Learn
import sklearn
from sklearn.model_selection import train_test_split

# Remoção de Warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 2 - Sistema de Recomendação App Contabilidade" --iversions

Author: Projeto 2 - Sistema de Recomendação App Contabilidade

sklearn        : 1.0.2
torch_geometric: 2.3.1
torch          : 2.0.0
numpy          : 1.21.5
pandas         : 1.4.4



## 2. Preparação do Conjunto de Dados

In [8]:
# Dataframe de usuários
users = pd.DataFrame({"user_id": [0, 1, 2, 3]})

# Dataframe de apps
aplicativos = pd.DataFrame({"app_id": [0, 1, 2]})

# Dataframe de avaliações (ratings)
ratings = pd.DataFrame({"user_id": [0, 1, 1, 2, 3],
                        "app_id": [0, 0, 1, 2, 2],
                        "rating": [4, 5, 3, 2, 4]})

In [9]:
# Visualiza dados de usuários
users

,user_id
0,0
1,1
2,2
3,3


In [10]:
# Visualiza dados de aplicativos
aplicativos

,app_id
0,0
1,1
2,2


In [11]:
# Visualia dados de ratings
ratings

,user_id,app_id,rating
0,0,0,4
1,1,0,5
2,1,1,3
3,2,2,2
4,3,2,4


In [12]:
# Converta os IDs dos aplicativos para evitar confusão com os IDs dos usuários
ratings["app_id"] += users.shape[0] - 1

In [13]:
# Visualiza as alterações em ratings
ratings

,user_id,app_id,rating
0,0,3,4
1,1,3,5
2,1,4,3
3,2,5,2
4,3,5,4
